In [5]:
import numpy as np
import os 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [6]:
# script to resize the images present in the dataset
# vgg16 model requires input images to be of size 224x224

train_dir = "Variant-a(Multiclass Classification)/train"
val_dir = "Variant-a(Multiclass Classification)/val"
test_dir = "Variant-a(Multiclass Classification)/test"
img_size = (224, 224)
batch_size = 15

saved_model_path = "saved_model/vgg16_plant_disease.keras"

In [7]:
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=45,        
    width_shift_range=0.5,     
    height_shift_range=0.5,
    shear_range=0.5,            
    zoom_range=0.5,             
    horizontal_flip=True,
    vertical_flip=True,         
    fill_mode='nearest'
)

val_test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

val_generator = datagen.flow_from_directory(
    val_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# Check class indices
print("Class Indices: ", train_generator.class_indices)

Found 3162 images belonging to 8 classes.
Found 902 images belonging to 8 classes.
Found 461 images belonging to 8 classes.
Class Indices:  {'Early_blight': 0, 'Healthy': 1, 'Late_blight': 2, 'Leaf Miner': 3, 'Magnesium Deficiency': 4, 'Nitrogen Deficiency': 5, 'Pottassium Deficiency': 6, 'Spotted Wilt Virus': 7}


In [8]:
if os.path.exists(saved_model_path):
    print("Loading the pre-trained model...")
    model = load_model(saved_model_path)
else:
    print("Training the model...")
    base_model = VGG16(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
    for layer in base_model.layers[:15]:  # freeze the first 15 layers
        layer.trainable = False

    for layer in base_model.layers[15:]:  # nnfreeze the rest
        layer.trainable = True
    
    model = Sequential([
        base_model,
        Flatten(),
        Dense(256, activation="relu"),
        BatchNormalization(),       
        Dropout(0.5),               
        Dense(128, activation="relu"),
        BatchNormalization(),    
        Dense(train_generator.num_classes, activation="softmax")
    ])
    
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss="categorical_crossentropy", metrics=["accuracy"])
    
    early_stop = EarlyStopping(
        monitor="val_loss",
        patience=7,
        restore_best_weights=True
    )
    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5, 
        patience=3, 
        min_lr=1e-6, 
        verbose=1
    )
    
    #steps_per_epoch = train_generator.samples // train_generator.batch_size
    #validation_steps = val_generator.samples // val_generator.batch_size
    
    #train 
    history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=25,
    callbacks=[early_stop, reduce_lr],
    verbose=1
    )
    
    #save 
    print("Saving the model...")
    model.save(saved_model_path)

Training the model...
Epoch 1/25
211/211 ━━━━━━━━━━━━━━━━━━━━ 479s 2s/step - accuracy: 0.1897 - loss: 2.4951 - val_accuracy: 0.2384 - val_loss: 4.4195 - learning_rate: 1.0000e-04
Epoch 2/25
211/211 ━━━━━━━━━━━━━━━━━━━━ 476s 2s/step - accuracy: 0.3499 - loss: 1.9041 - val_accuracy: 0.2871 - val_loss: 2.2565 - learning_rate: 1.0000e-04
Epoch 3/25
211/211 ━━━━━━━━━━━━━━━━━━━━ 468s 2s/step - accuracy: 0.4175 - loss: 1.6326 - val_accuracy: 0.4002 - val_loss: 3.3687 - learning_rate: 1.0000e-04
Epoch 4/25
211/211 ━━━━━━━━━━━━━━━━━━━━ 424s 2s/step - accuracy: 0.4699 - loss: 1.5264 - val_accuracy: 0.3803 - val_loss: 2.2124 - learning_rate: 1.0000e-04
Epoch 5/25
211/211 ━━━━━━━━━━━━━━━━━━━━ 420s 2s/step - accuracy: 0.5087 - loss: 1.4412 - val_accuracy: 0.4701 - val_loss: 1.5813 - learning_rate: 1.0000e-04
Epoch 6/25
211/211 ━━━━━━━━━━━━━━━━━━━━ 411s 2s/step - accuracy: 0.5257 - loss: 1.3914 - val_accuracy: 0.5787 - val_loss: 1.2156 - learning_rate: 1.0000e-04
Epoch 7/25
211/211 ━━━━━━━━━━━━━━━━━

KeyboardInterrupt: 

In [5]:
print("Evaluating the model...")
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {test_accuracy:.4f}")

In [ ]:
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.legend()
plt.title("Accuracy")

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.legend()
plt.title("Loss")
plt.show()

In [ ]:
#load model with load_model()